In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE47183'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"In silico nano-dissection: defining cell type specificity at transcriptional level in human disease (glomeruli)"
!Series_summary	"To identify genes with cell-lineage-specific expression not accessible by experimental micro-dissection, we developed a genome-scale iterative method, in-silico nano-dissection, which leverages high-throughput functional-genomics data from tissue homogenates using a machine-learning framework."
!Series_summary	"This study applied nano-dissection to chronic kidney disease and identified transcripts specific to podocytes, key cells in the glomerular filter responsible for hereditary proteinuric syndromes and acquired CKD. In-silico prediction accuracy exceeded predictions derived from fluorescence-tagged-murine podocytes, identified genes recently implicated in hereditary glomerular disease and predicted genes significantly correlated with kidney function. The nano-dissection method is broadly applicable to define lineage

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if the dataset contains gene expression data
is_gene_available = True

# Check availability of variables in the sample characteristics dictionary
sample_characteristics_dict = {
    0: ['tissue: Glomeruli from kidney biopsy'],
    1: [
        'disease status: Diabetic Nephropathy', 
        'disease status: Minimal Change Disease', 
        'disease status: Thin Membrane Disease', 
        'disease status: Tumor Nephrectomy', 
        'disease status: Focal and Segmental Glomerulosclerosis', 
        'disease status: Membranous Glomerulonephritis'
    ]
}

# 'Chronic_kidney_disease' can be inferred from 'disease status' related terms.
if len(set(sample_characteristics_dict[1])) > 1:
    trait_row = 1

# No direct mention of age and gender in the sample characteristics
age_row = None
gender_row = None

# Data type conversion functions
def convert_trait(value):
    if "disease status:" in value:
        return 1  # Considering presence of disease as trait
    return None  # If no 'disease status' in the value

def convert_age(value):
    try:
        age_value = int(value.strip())
        return age_value
    except ValueError:
        return None

def convert_gender(value):
    lower_val = value.strip().lower()
    if lower_val == 'female':
        return 0
    elif lower_val == 'male':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE47183', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if clinical data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE47183.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1146269': [1], 'GSM1146270': [1], 'GSM1146271': [1], 'GSM1146272': [1], 'GSM1146273': [1], 'GSM1146274': [1], 'GSM1146275': [1], 'GSM1146276': [1], 'GSM1146277': [1], 'GSM1146278': [1], 'GSM1146279': [1], 'GSM1146280': [1], 'GSM1146281': [1], 'GSM1146282': [1], 'GSM1146283': [1], 'GSM1146284': [1], 'GSM1146285': [1], 'GSM1146286': [1], 'GSM1146287': [1], 'GSM1146288': [1], 'GSM1146289': [1], 'GSM1146290': [1], 'GSM1146291': [1], 'GSM1146292': [1], 'GSM1146293': [1], 'GSM1146294': [1], 'GSM1146295': [1], 'GSM1146296': [1], 'GSM1146297': [1], 'GSM1146298': [1], 'GSM1146299': [1], 'GSM1146300': [1], 'GSM1146301': [1], 'GSM1146302': [1], 'GSM1146303': [1], 'GSM1146304': [1], 'GSM1146305': [1], 'GSM1146306': [1], 'GSM1146307': [1], 'GSM1146308': [1], 'GSM1146309': [1], 'GSM1146310': [1], 'GSM1146311': [1], 'GSM1146312': [1], 'GSM1146313': [1], 'GSM1146314': [1], 'GSM1146315': [1], 'GSM1146316': [1], 'GSM1146317': [1], 'GSM1146318': [1], 'GSM1146319': [1], 'GSM1146320': [1], 'GSM1146321

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['10', '100', '1000', '10000', '10001', '10002', '10003', '10004',
       '10005', '10006', '10007', '10009', '1001', '10010', '10013', '10014',
       '10015', '10016', '10017', '10018'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '9', '10', '12'], 'GeneID': ['1', '2', '9', '10', '12'], 'ORF': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Description': ['alpha-1-Bglycoprotein', 'alpha-2-macroglobulin', 'N-acetyltransferase1(arylamineN-acetyltransferase)', 'N-acetyltransferase2(arylamineN-acetyltransferase)', 'serpinpeptidaseinhibitor,cladeA(alpha-1antiproteinase,antitrypsin),member3'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Chronic_kidney_disease/gene_data/GSE47183.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Chronic_kidney_disease')

# 4. Save the cohort information.
save_cohort_info('GSE47183', './preprocessed/Chronic_kidney_disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Chronic_kidney_disease/GSE47183.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Chronic_kidney_disease':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Chronic_kidney_disease' in this dataset is severely biased.

